In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pycaret
from pycaret.classification import *
from pycaret.regression import *
from pycaret.datasets import get_data

In [3]:
data_pm = pd.read_csv('data_predict.csv')
data_pm

,DATETIMEDATA,PM25,O3,WS,TEMP,RH,WD
0,2024-01-01 00:00:00,16.5,16.000000,1,27.0,79.0,115.0
1,2024-01-01 01:00:00,33.6,11.044944,1,27.0,82.0,97.0
2,2024-01-01 02:00:00,34.2,11.044944,1,26.0,84.0,96.0
3,2024-01-01 03:00:00,20.1,11.044944,1,26.0,82.0,108.0
4,2024-01-01 04:00:00,16.5,1.000000,1,27.0,79.0,106.0
...,...,...,...,...,...,...,...
1672,2024-03-10 16:00:00,25.3,30.000000,1,31.0,53.0,194.0
1673,2024-03-10 17:00:00,23.9,28.000000,1,30.0,57.0,214.0
1674,2024-03-10 18:00:00,24.1,27.000000,1,29.0,63.0,202.0
1675,2024-03-10 19:00:00,28.6,4.000000,1,29.0,64.0,138.0


In [4]:
data_pm['DATETIMEDATA'] = pd.to_datetime(data_pm['DATETIMEDATA'])

In [5]:
regression_setup = setup(data_pm, target='RH', session_id=123, date_features=['DATETIMEDATA'], normalize=True)

,Description,Value
0,Session id,123
1,Target,RH
2,Target type,Regression
3,Original data shape,"(1677, 7)"
4,Transformed data shape,"(1677, 9)"
5,Transformed train set shape,"(1173, 9)"
6,Transformed test set shape,"(504, 9)"
7,Numeric features,5
8,Date features,1
9,Preprocess,True


In [6]:
best = compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,00:38:01
Status,. . . . . . . . . . . . . . . . . .,Loading Estimator
Estimator,. . . . . . . . . . . . . . . . . .,Linear Regression


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,1.9753,6.7565,2.5893,0.9492,0.0380,0.0292,0.3500
lightgbm,Light Gradient Boosting Machine,2.0949,7.5420,2.7395,0.9435,0.0407,0.0311,0.0920
gbr,Gradient Boosting Regressor,2.1954,8.1669,2.8512,0.9386,0.0420,0.0326,0.0210
et,Extra Trees Regressor,2.1740,8.4202,2.8924,0.9365,0.0432,0.0326,0.0410
rf,Random Forest Regressor,2.2750,9.2951,3.0391,0.9299,0.0451,0.0339,0.0670
ada,AdaBoost Regressor,3.0065,13.9825,3.7330,0.8951,0.0541,0.0442,0.0150
dt,Decision Tree Regressor,2.9255,15.9974,3.9833,0.8792,0.0582,0.0433,0.0080
knn,K Neighbors Regressor,3.0959,17.0048,4.1151,0.8719,0.0601,0.0459,0.0110
lar,Least Angle Regression,3.3580,17.6602,4.1975,0.8671,0.0599,0.0492,0.0070
br,Bayesian Ridge,3.3579,17.6608,4.1976,0.8671,0.0599,0.0492,0.0070


In [7]:
model = create_model('catboost')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.2535,9.3393,3.0560,0.9301,0.0431,0.0330
1,1.8413,6.0869,2.4672,0.9593,0.0367,0.0272
2,2.0553,7.2244,2.6878,0.9403,0.0385,0.0300
3,2.0569,7.0872,2.6622,0.9436,0.0400,0.0310
4,1.9479,6.4109,2.5320,0.9525,0.0381,0.0293
5,1.9526,6.6225,2.5734,0.9533,0.0377,0.0289
6,1.8118,5.8812,2.4251,0.9591,0.0349,0.0261
7,2.2927,8.2812,2.8777,0.9389,0.0425,0.0337
8,1.7416,5.2630,2.2941,0.9568,0.0351,0.0269


In [8]:
tuned_model = tune_model(model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.3869,10.4162,3.2274,0.9221,0.0462,0.0353
1,2.0035,6.7586,2.5997,0.9548,0.0393,0.0299
2,2.1265,7.9459,2.8188,0.9344,0.0405,0.0311
3,2.1934,8.1470,2.8543,0.9351,0.0424,0.0330
4,2.1185,7.3209,2.7057,0.9458,0.0403,0.0316
5,2.0164,7.1241,2.6691,0.9498,0.0394,0.0298
6,1.7478,5.5645,2.3589,0.9613,0.0344,0.0253
7,2.2972,8.2193,2.8669,0.9393,0.0429,0.0340
8,1.9121,6.8564,2.6185,0.9437,0.0410,0.0298


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [9]:
model_ensem20 = ensemble_model(tuned_model, n_estimators=20)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.3888,10.4549,3.2334,0.9218,0.0459,0.0353
1,1.9306,6.7106,2.5905,0.9552,0.0392,0.0288
2,2.0812,7.4077,2.7217,0.9388,0.0392,0.0305
3,2.1766,7.8436,2.8006,0.9376,0.0416,0.0326
4,2.0099,6.9283,2.6322,0.9487,0.0396,0.0304
5,2.0984,7.7850,2.7902,0.9452,0.0415,0.0313
6,1.7236,5.2970,2.3015,0.9632,0.0334,0.0249
7,2.3079,8.5762,2.9285,0.9367,0.0432,0.0338
8,1.8469,5.5334,2.3523,0.9546,0.0359,0.0285


In [10]:
last_time = data_pm['DATETIMEDATA'].max()
start_date = last_time + pd.DateOffset(hours=1)
end_date = start_date + pd.DateOffset(days=12, hours=23)  
next_week_hours = pd.date_range(start=start_date, end=end_date, freq='D')

In [11]:
next_data = pd.DataFrame({'DATETIMEDATA': next_week_hours})
next_data['O3'] = 0
next_data['WS'] = 0
next_data['TEMP'] = 0
next_data['PM25'] = 0
next_data['WD'] = 0

In [12]:
model_predictions = predict_model(model_ensem20, data=next_data)

In [13]:
mpdf = pd.DataFrame(model_predictions, columns=['DATETIMEDATA', 'prediction_label'])
mpdf = mpdf.rename(columns={'prediction_label': 'RH'})

print(mpdf)

          DATETIMEDATA         RH
0  2024-03-10 21:00:00  81.592331
1  2024-03-11 21:00:00  80.767084
2  2024-03-12 21:00:00  80.582902
3  2024-03-13 21:00:00  80.829566
4  2024-03-14 21:00:00  82.498247
5  2024-03-15 21:00:00  81.939829
6  2024-03-16 21:00:00  81.779762
7  2024-03-17 21:00:00  81.303783
8  2024-03-18 21:00:00  81.476819
9  2024-03-19 21:00:00  82.081513
10 2024-03-20 21:00:00  82.172485
11 2024-03-21 21:00:00  82.044472
12 2024-03-22 21:00:00  81.616418


In [14]:
prediction = pd.read_csv('new_model_predict.csv')
prediction

,DATETIMEDATA,PM25
0,2024-03-10,22.759762
1,2024-03-11,22.821399
2,2024-03-12,22.838767
3,2024-03-13,22.697659
4,2024-03-14,22.495483
5,2024-03-15,22.091482
6,2024-03-16,21.910590
7,2024-03-17,21.791230
8,2024-03-18,21.578651
9,2024-03-19,21.484950


In [15]:
merged_data = pd.concat([mpdf, prediction['PM25']], axis=1)
merged_data

,DATETIMEDATA,RH,PM25
0,2024-03-10 21:00:00,81.592331,22.759762
1,2024-03-11 21:00:00,80.767084,22.821399
2,2024-03-12 21:00:00,80.582902,22.838767
3,2024-03-13 21:00:00,80.829566,22.697659
4,2024-03-14 21:00:00,82.498247,22.495483
5,2024-03-15 21:00:00,81.939829,22.091482
6,2024-03-16 21:00:00,81.779762,21.910590
7,2024-03-17 21:00:00,81.303783,21.791230
8,2024-03-18 21:00:00,81.476819,21.578651
9,2024-03-19 21:00:00,82.081513,21.484950


In [16]:
merged_data.to_csv('new_model_predict.csv', index=False)

In [17]:
evaluate_model(model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…